In [1]:
from src.retriever.base_retriever import FaissStore

image_store = FaissStore(
    index_path="D:/BookInsight/data/vectorstores/image_faiss/shard_0/index.faiss",
    meta_path="D:/BookInsight/data/vectorstores/image_faiss/shard_0/index.pkl",
    modality="image"
)

image_store.load()
image_store.info()


📦 Loading FAISS index from D:/BookInsight/data/vectorstores/image_faiss/shard_0/index.faiss ...
✅ Loaded FAISS index with 4882 vectors.
📘 Loading metadata from D:/BookInsight/data/vectorstores/image_faiss/shard_0/index.pkl ...
✅ Metadata rows match FAISS index: 4882 entries.
🎯 Loaded IMAGE store successfully!
 FAISSStore (image)
• Index path : D:/BookInsight/data/vectorstores/image_faiss/shard_0/index.faiss
• Meta path  : D:/BookInsight/data/vectorstores/image_faiss/shard_0/index.pkl
• Vectors    : 4882


In [2]:
from importlib import reload
from src.retriever import base_retriever
reload(base_retriever)

from src.retriever.base_retriever import FaissStore

image_store = FaissStore(
    index_path="D:/BookInsight/data/vectorstores/image_faiss/shard_0/index.faiss",
    meta_path="D:/BookInsight/data/vectorstores/image_faiss/shard_0/index.pkl",
    modality="image"
)
image_store.load()


📦 Loading FAISS index from D:/BookInsight/data/vectorstores/image_faiss/shard_0/index.faiss ...
✅ Loaded FAISS index with 4882 vectors.
📘 Loading metadata from D:/BookInsight/data/vectorstores/image_faiss/shard_0/index.pkl ...
✅ Metadata rows match FAISS index: 4882 entries.
🎯 Loaded IMAGE store successfully!


In [3]:
import numpy as np


query_vec = np.random.randn(1, image_store.index.d)
query_vec = query_vec / np.linalg.norm(query_vec, axis=1, keepdims=True)

results = image_store.search(query_vec, k=5)
for r in results:
    print(f"Rank {r['rank']}: score={r['score']:.4f}, title={r['metadata'].get('title')}")


Rank 1: score=1.7275, title=Pool Party
Rank 2: score=1.7563, title=Filthy Rich
Rank 3: score=1.7753, title=Hollywood Screenwriting Directory Spring 2013: A Specialized Resource for Discovering Where & How to Sell Your Screenplay
Rank 4: score=1.8070, title=His Model Student
Rank 5: score=1.8191, title=Creating Musical Theatre: Conversations with Broadway Directors and Choreographers (Performance Books)


In [4]:
from src.retriever.text_image_retriever import TextImageRetriever
retriever = TextImageRetriever(
    text_index_path="D:/BookInsight/data/vectorstores/text_faiss/shard_0/index.faiss",
    text_meta_path="D:/BookInsight/data/vectorstores/text_faiss/shard_0/index.pkl",
    image_index_path="D:/BookInsight/data/vectorstores/image_faiss/shard_0/index.faiss",
    image_meta_path="D:/BookInsight/data/vectorstores/image_faiss/shard_0/index.pkl",
)
retriever.load_stores()


 Loading text FAISS store...
📦 Loading FAISS index from D:/BookInsight/data/vectorstores/text_faiss/shard_0/index.faiss ...
✅ Loaded FAISS index with 88956 vectors.
📘 Loading metadata from D:/BookInsight/data/vectorstores/text_faiss/shard_0/index.pkl ...
⚠️ Detected LangChain FAISS tuple — extracting InMemoryDocstore...
✅ Metadata rows match FAISS index: 88956 entries.
🎯 Loaded TEXT store successfully!

 Loading image FAISS store...
📦 Loading FAISS index from D:/BookInsight/data/vectorstores/image_faiss/shard_0/index.faiss ...
✅ Loaded FAISS index with 4882 vectors.
📘 Loading metadata from D:/BookInsight/data/vectorstores/image_faiss/shard_0/index.pkl ...
✅ Metadata rows match FAISS index: 4882 entries.
🎯 Loaded IMAGE store successfully!


In [5]:
query = "children's fantasy about dragons"
text_results = retriever.retrieve_text(query, k=5)

for r in text_results:
    print(f"({r['rank']}) {r['metadata'].get('title')} | score={r['score']:.4f}")


Loading BGE encoder (BAAI/bge-small-en-v1.5)...
(1) None | score=0.4183
(2) None | score=0.4572
(3) None | score=0.4966
(4) None | score=0.5086
(5) None | score=0.5230


In [6]:
import pandas as pd
from src.retriever.base_retriever import FaissStore

meta = pd.read_pickle("D:/BookInsight/data/vectorstores/text_faiss/shard_0/index.pkl")
print(type(meta))
if isinstance(meta, pd.DataFrame):
    print(meta.columns[:10])
elif isinstance(meta, tuple):
    print(type(meta[0]), getattr(meta[0], "columns", None))
else:
    print("⚠️ Metadata không phải DataFrame:", type(meta))


<class 'tuple'>
<class 'langchain_community.docstore.in_memory.InMemoryDocstore'> None


In [7]:
import importlib
from src.retriever import text_image_retriever
importlib.reload(text_image_retriever)

from src.retriever.text_image_retriever import TextImageRetriever


In [8]:
query = "children's fantasy about dragons"
text_results = retriever.retrieve_text(query, k=5)
image_results = retriever.retrieve_image_by_text(query, k=5)
fused = retriever.fuse_results(text_results, image_results, method="rrf")

for r in fused:
    print(f"{r['rank']:>2}. [{r['source']}] {r['metadata'].get('title') or r['metadata'].get('content')} | {r['fused_score']:.4f}")


🧠 Loading CLIP text encoder (clip-ViT-B-16)...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


[INFO]  Gộp 10 kết quả cuối cùng từ 10 kết quả gốc.
[text] the prince and his dragon: u.s. english edition - magical rhyming bedtime story - picture book / beginner reader, about  | score=0.0909 | count=1
[image] DRAGONS KINGS DRAGONLING 6 | score=0.0909 | count=1
     🖼 https://m.media-amazon.com/images/I/51ACFGHP50L._SX330_BO1,204,203,200_.jpg
[text] dragon riders paperback – february 23, 2010 dragon riders is an amazing adventure about three children caught up in a ti | score=0.0833 | count=1
[image] Beware the Crystal Dragon | score=0.0833 | count=1
     🖼 https://m.media-amazon.com/images/I/51DdR4WcnEL._SX300_BO1,204,203,200_.jpg
[text] the language of spells: (fantasy middle grade novel, magic and wizard book for middle school kids) hardcover – june 26,  | score=0.0769 | count=1
[image] The Princess and the Dragon | score=0.0769 | count=1
     🖼 https://m.media-amazon.com/images/I/51LsO+0CHYL._SX473_BO1,204,203,200_.jpg
[text] dragon fantasy - mosaic color by number -enchanted co

In [9]:
query = "romantic fantasy with vampires"
text_results = retriever.retrieve_text(query, k=5)
image_results = retriever.retrieve_image_by_text(query, k=5)
fused = retriever.fuse_results(text_results, image_results, method="rrf")

for r in fused:
    print(f"{r['rank']:>2}. [{r['source']}] {r['metadata'].get('title') or r['metadata'].get('content')} | {r['fused_score']:.4f}")


[INFO]  Gộp 10 kết quả cuối cùng từ 10 kết quả gốc.
[text] beneath unknown  books > romance > fantasy unknown unknown | score=0.0909 | count=1
[image] Zane: A Sin City Sanguines Novel | score=0.0909 | count=1
     🖼 https://m.media-amazon.com/images/I/41XYCDuuCoL._SX311_BO1,204,203,200_.jpg
[text] guarded unknown  books > romance > fantasy unknown unknown | score=0.0833 | count=1
[image] Touched:: The Hunter Legacy | score=0.0833 | count=1
     🖼 https://m.media-amazon.com/images/I/41RejHig4zL._SX331_BO1,204,203,200_.jpg
[text] twilight hardcover – october 5, 2005 amazon.com review softly he brushed my cheek, then held my face between his marble  | score=0.0769 | count=1
[image] Taste of Darkness (The Siorai Legacy III) | score=0.0769 | count=1
     🖼 https://m.media-amazon.com/images/I/510l3Db5vkL.jpg
[text] vampire of my dreams paperback – september 21, 2010 review when the impossible is real, and the real is imaginary, bound | score=0.0714 | count=1
[image] The House of Haunted Drea

In [11]:
query = "fairy tale with magical animals and kingdoms"
text_results = retriever.retrieve_text(query, k=5)
image_results = retriever.retrieve_image_by_text(query, k=5)
fused = retriever.fuse_results(text_results, image_results, method="rrf")

for r in fused:
    print(f"{r['rank']:>2}. [{r['source']}] {r['metadata'].get('title') or r['metadata'].get('content')} | {r['fused_score']:.4f}")


[INFO]  Gộp 10 kết quả cuối cùng từ 10 kết quả gốc.
[text] more english fairy tales (forgotten books) unknown book description: a fairy tale or fairy story is a fictional story th | score=0.0909 | count=1
[image] The Princess and the Dragon | score=0.0909 | count=1
     🖼 https://m.media-amazon.com/images/I/51LsO+0CHYL._SX473_BO1,204,203,200_.jpg
[text] fabulous creatures, mythical monsters, and animal power symbols: a handbook unknown from school library journal grade 9  | score=0.0833 | count=1
[image] Best Children's Songs Ever | score=0.0833 | count=1
     🖼 https://m.media-amazon.com/images/I/51nLU9PMJZL._SX373_BO1,204,203,200_.jpg
[text] the king with horse's ears and other irish folktales (folktales of the world) hardcover – february 3, 2009 from school l | score=0.0769 | count=1
[image] DRAGONS KINGS DRAGONLING 6 | score=0.0769 | count=1
     🖼 https://m.media-amazon.com/images/I/51ACFGHP50L._SX330_BO1,204,203,200_.jpg
[text] horse: a novel unknown  books > literature & fiction